# Multichannel Potentials


In [1]:
from data_gen import get_data
import numpy as np

#### Load the molecule through the get_data function

In [2]:
mol = get_data("/Users/lucabrodoloni/Desktop/Stage/Vettore_download/data/dsgdb9nsd_106437.xyz")

In [3]:
n_atoms = mol[0]
difference_energy = mol[5]
frame = mol[2]

In [4]:

def multi_potential(
    atomic_info: np.ndarray,
    gamma: float = 0.36,
    alpha: float = 6.0,
    n_points: int = 32,
    physic_length: float = 12.0
):
    V_pot = np.zeros((n_points, n_points, n_points))
    grid_space = physic_length / n_points
    n_atoms = atomic_info.shape[0]
    offset = physic_length / 2
    for i in range(n_points):
        for j in range(n_points):
            for k in range(n_points):
                V_term_space = np.zeros(n_atoms)
                for l in range(n_atoms):
                    V_term_space[l] = alpha * np.exp(
                        (-1.0 / (2 * (gamma) ** 2))
                        * (
                            (((i * grid_space) - offset + offset_x) - atomic_info[l, 0]) ** 2
                            + (((j * grid_space) - offset + offset_y) - atomic_info[l, 1]) ** 2
                            + (((k * grid_space) - offset + offset_z) - atomic_info[l, 2]) ** 2
                        )
                    )
                # sum the terms in the function and make it single precision.
                V_pot[i, j, k] = np.single(np.sum(V_term_space))
    return V_pot

#### Compute the global x, y, z offsets for each molecule

In [5]:
offset_x = (abs(max(frame[:, 1])) - abs(min(frame[:, 1]))) / 2
offset_y = (abs(max(frame[:, 2])) - abs(min(frame[:, 2]))) / 2
offset_z = (abs(max(frame[:, 3])) - abs(min(frame[:, 3]))) / 2
print(offset_x, offset_y, offset_z)
print(frame)


0.394933819770813 -1.1921765804290771 1.2904892265796661
[[ 6.         -0.05013775  1.34459448  0.01464414]
 [ 8.          0.01573426 -0.06770165  0.09227195]
 [ 6.          0.68811589 -0.5427258   1.18952596]
 [ 6.          2.20329237 -0.39400503  1.55221486]
 [ 6.          1.57768512 -1.31476092  2.63546371]
 [ 6.          0.37257096 -0.33488858  2.70889735]
 [ 6.          0.84100354 -2.07025623  1.47145867]
 [ 6.         -0.32675502 -2.87656641  1.75876796]
 [ 6.         -1.28264928 -3.57081938  1.98502874]
 [ 1.          0.95040828  1.79408598 -0.05411734]
 [ 1.         -0.61120862  1.58645272 -0.890109  ]
 [ 1.         -0.56907421  1.77342832  0.88297451]
 [ 1.          2.92054796 -0.85163099  0.86590821]
 [ 1.          2.53419471  0.58896935  1.89416087]
 [ 1.          2.0896287  -1.78451002  3.47108746]
 [ 1.          0.5856564   0.65227073  3.12583756]
 [ 1.         -0.57129508 -0.7416361   3.07358289]
 [ 1.          1.51188791 -2.58296466  0.77504134]
 [ 1.         -2.13068032

In [6]:
# potenziali multicanale
# la forma del tensore è 5 x 32 x 32 x 32 dove 5 è il numero di canali (specie atomiche)
multichannel_potential = np.zeros((5, 32, 32, 32))
H_coord = []
C_coord = []
N_coord = []
O_coord = []
F_coord = []
for i in range(0, len(frame)):
    if frame[i][0] == 1.0:
        H_coord.append(frame[i][1:4])
    elif frame[i][0] == 6.0:
        C_coord.append(frame[i][1:4])
    elif frame[i][0] == 7.0:
        N_coord.append(frame[i][1:4])
    elif frame[i][0] == 8.0:
        O_coord.append(frame[i][1:4])
    elif frame[i][0] == 9.0:
        F_coord.append(frame[i][1:4])
if len(H_coord) != 0:
       H_coord = np.array(H_coord).reshape(len(H_coord), 3)
       H_pot = multi_potential(atomic_info=H_coord)
       multichannel_potential[0] = H_pot
else:
    multichannel_potential[0] = np.zeros((32, 32, 32))
if len(C_coord) != 0:
       C_coord = np.array(C_coord).reshape(len(C_coord), 3)
       C_pot = multi_potential(atomic_info=C_coord)
       multichannel_potential[1] = C_pot
else:
    multichannel_potential[1] = np.zeros((32, 32, 32))
if len(N_coord) != 0:
       N_coord = np.array(N_coord).reshape(len(N_coord), 3)
       N_pot = multi_potential(atomic_info=N_coord)
       multichannel_potential[2] = N_pot
else:
    multichannel_potential[2] = np.zeros((32, 32, 32))
if len(O_coord) != 0:
       O_coord = np.array(O_coord).reshape(len(O_coord), 3)
       O_pot = multi_potential(atomic_info=O_coord)
       multichannel_potential[3] = O_pot
else:
    multichannel_potential[3] = np.zeros((32, 32, 32))
if len(F_coord) != 0:
       F_coord = np.array(F_coord).reshape(len(F_coord), 3)
       F_pot = multi_potential(atomic_info=F_coord)
       multichannel_potential[4] = F_pot
else:
    multichannel_potential[4] = np.zeros((32, 32, 32))
# print(f"Hydrogen coordinates: \n{len(H_coord)}")
# H_coord = np.array(H_coord).reshape(len(H_coord), 3)
# print(H_coord)
# print(f"Carbon coordinates: \n{C_coord}")
# print(f"Nitrogen coordinates: \n{N_coord}")
# print(f"Oxigen coordinates: \n{O_coord}")
# print(f"Fluorine coordinates: \n{F_coord}")


In [7]:
# multichannel_potential[0, ...].shape
# multichannel_potential[1, ...].shape
# multichannel_potential[2, ...].shape
# multichannel_potential[3, ...].shape
# multichannel_potential[4, ...].shape
H_potential = multi_potential(atomic_info=H_coord)
C_potential = multi_potential(atomic_info=C_coord)
# N_potential = multi_potential(atomic_info=N_coord)
# O_potential = multi_potential(atomic_info=O_coord)
# F_potential = multi_potential(atomic_info=F_coord)
# H_potential.mean()



In [8]:
print(f"Max C: {C_potential.max()}, max H: {H_potential.max()}")


Max C: 5.584587097167969, max H: 5.695024013519287


In [9]:
multichannel_potential[0, ...] = H_potential
multichannel_potential[1, ...] = C_potential
multichannel_potential

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [10]:
for i in range(len(multichannel_potential)):
    print(multichannel_potential[i].max())
    

5.695024013519287
5.584587097167969
0.0
5.876776218414307
0.0


In [11]:
pot = np.load("/Users/lucabrodoloni/Desktop/multichannel_potential.npy")

In [14]:
for i in range(len(pot)):
    print(pot[i].max())

5.241522789001465
5.4652485847473145
0.0
5.998526573181152
0.0


In [38]:
from multi_chann_script import main

In [39]:
multi_pot = main("/Users/lucabrodoloni/Desktop/Stage/Vettore_download/dati_puliti/train/dsgdb9nsd_000017.xyz")

In [40]:
for i in range(len(multi_pot)):
    print(multi_pot[i].max())


5.241522789001465
5.4652485847473145
0.0
5.998526573181152
0.0
Diffenerce energy: <class 'numpy.ndarray'>
